In [ ]:
#coding:utf-8
import os 
import time 
import random
import jieba
import sklearn 
from sklearn.naive_bayes import MultinomialNB
import numpy as np 
import pylab as pl
import matplotlib.pyplot as plt

In [ ]:
def remove_deplicated_words(words):
    words_set = set()  # 利用set 的属性仅存储单一的元素
    with open(words,'r',encoding = 'utf-8') as readwords:
        for line in readwords.readlines():
            word  = line.strip()  # word is string 
            if len(word)>0 and word not in words_set: # 没有出现再现有的words 集合set 中则，增加进来
                words_set.add(word)
    return words_set

In [ ]:
    folder_path = './data/Database/SogouC/Sample'
    folder_list = os.listdir(folder_path)              # listdir - 用于列出该路劲下所有的文件及文件夹
    data_list   = []
    label_list  = []
    index = 0
    for folder in folder_list:
        new_folder_path = os.path.join(folder_path,folder)
        files           = os.listdir(new_folder_path)  # 读取每个文件夹下面的全部文件
        index += 1
        label_list.append(folder)
        #print("index = %s: files= %s: it's label = %s " %(index,files,label_list) )

In [ ]:
def Examples_Generated(folder_path,test_dataset_percentage = 0.20):
    folder_list = os.listdir(folder_path)              # listdir - 用于列出该路劲下所有的文件及文件夹
    data_list   = []
    label_list  = []
    
    for folder in folder_list:
        new_folder_path = os.path.join(folder_path,folder)
        files           = os.listdir(new_folder_path)  # 读取每个文件夹下面的全部文件
        
        # Reading Files
        jindex =  1 # index initialization 
        for file in files:
            # worry about memory if >100:
            if jindex >= 100:
                print("jindex >= 100,Break Now!")
            
            # reading file from detail file path 
            with open(os.path.join(new_folder_path,file),'r',encoding = 'utf-8') as open_handel:
                read_rawdata = open_handel.read()
            
            # jieba cut - 分词
            rawdata_cut      = jieba.cut(read_rawdata,cut_all = False)  # cut 精简model 
            rawdata_cut_list = list(rawdata_cut)
            
            data_list.append(rawdata_cut_list) # data 
            label_list.append(folder) # folder name is the label of the data in this folder
            # label_list.append(folder.decode('utf-8'))
    
    # divide the dataset into train and test data manual 
    data_label_list = list(zip(data_list,label_list))# 将两个list以元祖的形式组合成list = [(data_list, label_list)]
    random.shuffle(data_label_list)  # 乱序
    
    # 手动区分 - Index_boundary 
    index_boundary = int(len(data_label_list)* test_dataset_percentage) 
    test_dataset   = data_label_list[0:index_boundary]
    train_dataset  = data_label_list[index_boundary: ]
    
    # 通过zip(*) 将原来组合的list 再重写解开成两个单独的list, one is data, other one is list 
    test_data,test_data_lable   = list(zip(*test_dataset))
    train_data,train_data_lable = list(zip(*train_dataset))
    
    # transform to list type
    train_data_lable = list(train_data_lable)
    test_data_lable  = list(test_data_lable)
    test_data  = list(test_data)
    train_data = list(train_data)
    
    # using sklearn train_test_split to split train and test data
    #from sklearn.model_selection import train_test_split
    #x_trian,x_test,y_train,y_test = train_test_split(train_dataset,test_dataset,random_state = 200)
    
    # 统计词频方法all_words_dict 中
    statics_wordsfrequency_dict  = {}
    for word in train_data:     # every single 'word' in train_data 
        #print("word = ", word)
        for character in word:  # every single 'character' in word
            #print("character  = ", character)
            if character in statics_wordsfrequency_dict:  # python3.x deleted has_key, has_key only used in python 2.x
                statics_wordsfrequency_dict[character] += 1   # dict[key] = value 赋值方法！
            else:
                statics_wordsfrequency_dict[character]  = 1

    # key = lambda x:x[1] - 按照每个元素的第二值即value 排序，reverse =True 逆序-降序排列
    order_statics_list = sorted(statics_wordsfrequency_dict.items(),key = lambda x:x[1], reverse =True) 
    all_words_list     = list(list(zip(*order_statics_list))[0]) # zip(*) 要转成list in python 3.x 
    
    # return 想要的结果
    return all_words_list, train_data, test_data, train_data_lable, test_data_lable  

In [ ]:
def feature_extraction (all_words_list,deletN, stopwords_set = set() ):
    # feature extraction 
    feature_words =  []
    n             =   1  # initialization n index 
    for t in range(deletN,len(all_words_list),1):
        if n>10000:      # feature _ words 的维数
            print("n is bigger than 10000,Break !")
            break
        # if all is digit, isdigit() return True 
        if not all_words_list[t].isdigit() and all_words_list[t] not in stopwords_set and 1< len(all_words_list[t])<5:
            feature_words.append(all_words_list[t])
            n += 1
    
    return feature_words

In [ ]:
def text_features(train_data,test_data,feature_words,flag = 'nltk'):
    def text_features(text,feature_words):
        text_words = set(text)
        ##---------
        if flag   == 'nltk':
            features = {word:  1 if word in text_words else 0 for word in feature_words }
            
        elif flag == 'sklearn':
            features = [1 if word in text_words else 0 for word in feature_words]
        else:
            features = []
        ##---------
        return features
    
    train_feature_list  =  [text_features(text,feature_words) for text in train_data]
    test_feature_list   =  [text_features(text,feature_words) for text in test_data]
    return train_feature_list, test_feature_list

In [ ]:
def text_model_classificaton(train_feature_list,test_feature_list,train_data_lable,test_data_lable,flag = 'nltk'):
    ## -----
    if flag == 'nltk':
        ## 使用nltk分类器
        train_list = zip(train_feature_list,train_data_lable)
        test_list  = zip(test_feature_list,test_data_lable)
        classifier = nltk.classify.NaiveBayesClassifier.train(train_list) # NaiveBayesClassifier  
        accuracy_test  = nltk.classify.accuracy(classifier,test_list)
    # -----
    elif flag == 'sklearn':
        print("train_feature_list is ",type(train_feature_list))
        print("train_data_lable is ",type(train_data_lable))
        classifier = MultinomialNB().fit(train_feature_list,train_data_lable)
        accuracy_test  = classifier.score(test_feature_list,test_data_lable)
    else:
        accuracy_test  = []
    return accuracy_test

In [ ]:
    train_data = ['thshesieeiiiiie ','hedd','thisedusiie']
    statics_wordsfrequency_dict  = {}
    for word in train_data:     # every single 'word' in train_data 
        #print("word = ", word)
        for character in word:  # every single 'character' in word
            #print("character  = ", character)
            if character in statics_wordsfrequency_dict:  # python3.x deleted has_key, has_key only used in python 2.x
                statics_wordsfrequency_dict[character] += 1   # dict[key] = value 赋值方法！
            else:
                statics_wordsfrequency_dict[character]  = 1

    # key = lambda x:x[1] - 按照每个元素的第二值即value 排序，reverse =True 逆序-降序
    order_statics_list = sorted(statics_wordsfrequency_dict.items(),key = lambda x:x[1], reverse =True) # 
    
    #statics_wordsfrequency_dict
    e = list(list(zip(*order_statics_list))[0])

In [ ]:
print("Starting ... ")

# Text pre-processing 
folder_path = "./data/Database/SogouC/Sample"
all_words_list,train_data,test_data,train_data_lable,test_data_lable = Examples_Generated(folder_path,test_dataset_percentage = 0.20)

print(len(all_words_list))
print(type(train_data))
print(type(test_data))
print(type(train_data_lable))
print(type(test_data_lable))

# Generated stop words
stopwords_txtfile = './data/stopwords_cn_in5NavBayesTextClassifier.txt'
stopwords_set     = remove_deplicated_words(stopwords_txtfile)

# Text Extraction and Classification 
flag     = 'sklearn'
deleteN  = 1000
accuracy_test_list = []
#for deleteN in deleteNs:
feature_words =   feature_extraction(all_words_list,deleteN,stopwords_set)
train_feature_list, test_feature_list = text_features(train_data,test_data,feature_words)

import numpy as np
print(np.array(train_feature_list).shape)
print(np.array(train_data_lable).shape)
print(np.array(test_feature_list).shape)
print(np.array(test_data_lable).shape)

accuracy_test      =  text_model_classificaton(train_feature_list,test_feature_list,train_data_lable,test_data_lable,flag)
accuracy_test_list = accuracy_test_list.append(accuracy_test)
print("Accuracy score is :" , accuracy_test_list)

# Result 
'''plt.figure()
plt.plot(deleteNs,accuracy_test_list)
plt.title("relationship between deleteNs and accuracy_test_list")
plt.xlabel("deleteNs")
plt.ylabel("accuracy_test_list")
plt.show()

print("Completed!")'''